### LangChain Expression Language (LCEL)

In [1]:
import os
import time
from dotenv import load_dotenv

# Import LangChain components
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough, RunnableLambda


In [4]:
'''     Env setup and Gemini model initialization     '''
load_dotenv() 

# Check if the API key is loaded
if "GOOGLE_API_KEY" not in os.environ:
    print("Error: GOOGLE_API_KEY not found in environment variables.")
    exit()

# models: ['gemini-2.5-pro', 'gemma-3-27b-it']
llm = ChatGoogleGenerativeAI(
    model="gemma-3-27b-it", 
    temperature=0.7, 
)

# StrOutputParser() simply extracts the string content from the AIMessage
parser = StrOutputParser()

### 1. simple invocation (basic prompt)

In [ ]:
# This is the most basic way to call the model
result = llm.invoke("Who was the first person to walk on the moon?")
print(result.content)


### 2. build chain with the prompt template (Sequential)

#### ---> basic

In [ ]:
# This is the most basic way to call the model

# Create a template for our prompt. The {topic} part is a variable.
template = "Tell me a short joke about {topic}."
prompt = PromptTemplate(template=template, input_variables=["topic"])

# Create the chain by piping the prompt to the language model
# The output of the prompt will be the input for the llm
chain = prompt | llm

# Invoke the chain by passing the value for our 'topic' variable
response = chain.invoke({"topic": "a programmer"})
print(response.content)


#### ---> adding output parser for cleaner output 

In [ ]:
# Rebuild the chain, adding the parser at the end
chain_with_parser = prompt | llm | parser

# Invoke the new chain
clean_response = chain_with_parser.invoke({"topic": "a cat"})

print(f"Type of response: {type(clean_response)}")
print("Response:")
print(clean_response)

#### ---> advance chain (passing through input)

In [ ]:
# What if you want to see both the original question and the answer?
# We can use RunnablePassthrough to pass the input topic along the chain.
print("\n=========        Chain with Passthrough       =========")

final_chain = (
    {"joke": prompt | llm | parser} 
    | RunnablePassthrough()
)

# The result is now a dictionary containing the generated joke
result_dict = final_chain.invoke({"topic": "a robot"})
print(result_dict)

### 3. sequential

### 4. parallel

In [8]:
# --- DEFINE THE INPUT TEXT ---
input_text = """
The new SuperGraphX 5000 is a revolutionary graphics card. 
It delivers breathtaking visuals and unparalleled performance, making every game an immersive experience.
However, its high price point of $1200 might be a deterrent for budget-conscious builders. 
The power consumption is also noticeably high, requiring a robust power supply.
"""

# --- BUILD THE PARALLEL CHAIN (Your exact idea, formalized) ---

# This is the canonical way to write what you proposed.
# We define a dictionary where each value is a separate chain.
# We add StrOutputParser to get clean string outputs.
parallel_chain = RunnableParallel({
    'summary': ChatPromptTemplate.from_template('Summarise this text in one sentence: {text}') | llm | parser,
    'translation': ChatPromptTemplate.from_template('Translate this text into French: {text}') | llm | parser,
    'sentiment': ChatPromptTemplate.from_template('What is the overall sentiment in this text? (positive, negative or neutral): {text}') | llm | parser,
    'keywords': ChatPromptTemplate.from_template('Extract 5 main keywords from this text, separated by commas: {text}') | llm | parser,
})

In [9]:
# --- EXECUTE THE CHAIN AND MEASURE TIME ---
start_time = time.time()
# The input dictionary key 'text' is passed to every prompt template in the parallel_chain
results = parallel_chain.invoke({'text': input_text})
end_time = time.time()

print(f"\nParallel execution took: {end_time - start_time:.2f} seconds")


# --- DISPLAY THE RESULTS ---
print("\n--- Results ---")
print(f"\n[Summary]:\n{results['summary']}")
print(f"\n[French Translation]:\n{results['translation']}")
print(f"\n[Sentiment]:\n{results['sentiment']}")
print(f"\n[Keywords]:\n{results['keywords']}")


Parallel execution took: 16.40 seconds

--- Results ---

[Summary]:
The SuperGraphX 5000 is a high-performing graphics card offering stunning visuals, but its $1200 price and high power consumption may limit its appeal to serious gamers with substantial budgets and capable systems.

[French Translation]:
Here are a few options for the translation, ranging from more literal to slightly more fluid. I've included notes on the nuances:

**Option 1 (More Literal):**

> La nouvelle SuperGraphX 5000 est une carte graphique révolutionnaire.
> Elle offre des visuels époustouflants et des performances inégalées, transformant chaque jeu en une expérience immersive.
> Cependant, son prix élevé de 1200 $ pourrait être un frein pour les constructeurs soucieux de leur budget.
> La consommation d'énergie est également notablement élevée, nécessitant une alimentation robuste.

* **Strengths:** Very accurate to the original meaning.
* **Weaknesses:**  A little bit stiff in places. "Un frein" is perfect

### 5. Role playing


In [ ]:
def format_prompt(variables):
    return prompt.format(**variables)

In [ ]:
role = """
    Dungeon & Dragons game master
"""

tone = "engaging and immersive"

template = """
    You are an expert {role}. I have this question {question}. I would like our conversation to be {tone}.
    
    Answer:
"""
prompt = PromptTemplate.from_template(template)

# Create the LCEL chain
roleplay_chain = (
    RunnableLambda(format_prompt)
    | llm 
    | StrOutputParser()
)

In [ ]:
# Create an interactive chat loop
while True:
    query = input("Question: ")
    
    if query.lower() in ["quit", "exit", "bye"]:
        print("Answer: Goodbye!")
        break
        
    response = roleplay_chain.invoke({"role": role, "question": query, "tone": tone})
    print("Answer: ", response)